### Build annotation file for iTol phylogenetic tree
1. strip color dataset for strain type
2. color gradient dataset for genome size
3. color gardient dataset for proteome size
4. strip color dataset for source origin


In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

os.chdir(r'D:\code\github\Unified_Yeast_GEMs_Database')


In [2]:
# load strain information
all_strain_info=pd.read_excel('data/1897_strains_info.xlsx',index_col=0)
all_strain_info.head()

df_geneMatrix=pd.read_csv(r'data/geneMatrix/pan1800_v2_blastp_50_70_geneMatrix.csv',index_col=0)
strainList=df_geneMatrix.columns.tolist()
strainList=[strain.replace('.fa','') for strain in strainList]
# only keep the strains in the strain information
all_strain_info=all_strain_info[all_strain_info.index.isin(strainList)]

all_strain_info['type_L3'].unique()
# 删除type_L3中两侧的空格
all_strain_info['type_L3']=all_strain_info['type_L3'].str.strip()
# replace all space in type_L3 into _
all_strain_info['type_L3']=all_strain_info['type_L3'].str.replace(' ','_')

all_strain_info['type_L3'].value_counts()

type_L3
Wine                       370
Tree                       187
Human                      172
Beer                       163
Soil                       130
Unknown                    107
Mantou                      98
Fruit                       88
Fermentation-grain          69
Sake                        54
Dairy                       46
Bakery                      42
Bioethanol                  39
Palm_wine                   39
Fermentation-unassigned     36
Industrial-unassigned       31
Distillery                  31
Insect                      21
Baijiu                      21
Lab                         20
Cider                       18
Flower                      14
Huangjiu                    10
Fermentation-fruit           5
Name: count, dtype: int64

#### 1. strip color dataset for strain type

In [11]:
def generate_contrast_colors(n):
    '''Automatically generate contrast colors'''
    hues = np.linspace(0, 1, n, endpoint=False)
    
    colors = [mcolors.hsv_to_rgb((h, 1.0, 1.0)) for h in hues]
    
    # 转换为十六进制格式
    color_hex = [mcolors.rgb2hex(color) for color in colors]
    
    return color_hex

df_straintype=all_strain_info[['type_L3']]
type_list=list(df_straintype['type_L3'].unique())
color_sets=generate_contrast_colors(n=len(type_list))
# type_to_color=dict(zip(type_list,color_sets))
type_to_color={'Wine':'#4c1130',
               'Palm_wine':'#a64d79',
               'Beer':'#20124d',
               'Cider':'#674ea7',
               'Distillery':'#d5a6bd',
                'Sake':'#8e7cc3',
                'Baijiu':'#073763',
                'Huangjiu':'#3d85c6',
                'Fruit':'#274e13',
               'Soil':'#38761d',
               'Tree':'#6aa84f',
               'Flower':'#b6d7a8',
               'Insect':'#8fce00',
                'Bioethanol':'#0040ff',
                'Industrial-unassigned':'#9fc5e8',
                'Bakery':'#7f6000',
                'Dairy':'#bf9000',
                'Mantou':'#ffd966',
                'Fermentation-fruit':'#b45f06',
               'Fermentation-grain':'#f9cb9c',
               'Fermentation-unassigned':'#fff2cc',
               'Human':'#ff0000',
               'Lab':'#990000',
               'Unknown':'#5b5b5b'}

# convert rgb to hex
type_to_color_hex={}
for type,color in type_to_color.items():
    type_to_color_hex[type]=mcolors.rgb2hex(mcolors.to_rgb(color))


df_straintype['color']=''
for type in type_list:
    df_straintype['color'][df_straintype['type_L3']==type]=type_to_color_hex[type]

df_straintype=df_straintype[['color','type_L3']]
df_straintype.head()

# write df_straintype behind the code/1.genome_collection/outputs/sce1900_strain_type_color_strip.txt and use space as separator
df_straintype.to_csv('code/1.genome_collection/outputs/sce1807_strain_type_L3_color_strip.txt',sep=' ')

# add content in sce1900_strain_type_color_strip0.txt to the head of sce1900_strain_type_color_strip.txt
with open('code/1.genome_collection/outputs/sce1900_strain_type_color_strip0.txt','r') as f:
    content=f.read()
    with open('code/1.genome_collection/outputs/sce1807_strain_type_L3_color_strip.txt','r+') as f1:
        original=f1.read()
        f1.seek(0)   # move the cursor to the head of file
        f1.write(content+'\n'+original)

C:\Users\wangh\AppData\Local\Temp\ipykernel_28088\1892176347.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_straintype['color']=''
C:\Users\wangh\AppData\Local\Temp\ipykernel_28088\1892176347.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_straintype['color'][df_straintype['type_L3']==type]=type_to_color_hex[type]
C:\Users\wangh\AppData\Local\Temp\ipykernel_28088\1892176347.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [ ]:
# # use genome_id column as index and choose type column as column in df_straintype
# df_straintype=all_strain_info[['genome_id','sub_type']]
# df_straintype.set_index('genome_id',inplace=True)

# # change all 0 value as unknown
# df_straintype.replace(0,'unknown',inplace=True)
# df_straintype=df_straintype[df_straintype.index.isin(strainIDlist)]

# # set different color for different strain type
# # total strain type: Wine Wild Fermentation Human Beer unknown Sake Bakery Bioethanol Soil Industrial Distillery Dairy Insect Lab Cider
# # color list: #ff0000 #ffa500 #ffff00 #008000 #00ffff #0000ff #ee82ee #800000 #808000 #008080 #000080 #4b0082 #ff69b4 #00ff7f #ffa07a #a9a9a9
# df_straintype['color']=''
# df_straintype.loc[df_straintype['sub_type']=='Wine','color']='#ff0000'
# df_straintype.loc[df_straintype['sub_type']=='Wild','color']='#ffa500'
# df_straintype.loc[df_straintype['sub_type']=='Fermentation','color']='#ffff00'
# df_straintype.loc[df_straintype['sub_type']=='Human','color']='#008000'
# df_straintype.loc[df_straintype['sub_type']=='Beer','color']='#00ffff'
# df_straintype.loc[df_straintype['sub_type']=='unknown','color']='#0000ff'
# df_straintype.loc[df_straintype['sub_type']=='Sake','color']='#ee82ee'
# df_straintype.loc[df_straintype['sub_type']=='Bakery','color']='#800000'
# df_straintype.loc[df_straintype['sub_type']=='Bioethanol','color']='#808000'
# df_straintype.loc[df_straintype['sub_type']=='Soil','color']='#008080'
# df_straintype.loc[df_straintype['sub_type']=='Industrial','color']='#000080'
# df_straintype.loc[df_straintype['sub_type']=='Distillery','color']='#4b0082'
# df_straintype.loc[df_straintype['sub_type']=='Dairy','color']='#ff69b4'
# df_straintype.loc[df_straintype['sub_type']=='Insect','color']='#00ff7f'
# df_straintype.loc[df_straintype['sub_type']=='Lab','color']='#ffa07a'
# df_straintype.loc[df_straintype['sub_type']=='Cider','color']='#a9a9a9'

# # change the order of columns
# df_straintype=df_straintype[['color','sub_type']]
# # write df_straintype behind the code/1.genome_collection/outputs/sce1900_strain_type_color_strip.txt and use space as separator
# df_straintype.to_csv('code/1.genome_collection/outputs/sce1900_strain_type_color_strip.txt',sep=' ')

# # add content in sce1900_strain_type_color_strip0.txt to the head of sce1900_strain_type_color_strip.txt
# with open('code/1.genome_collection/outputs/sce1900_strain_type_color_strip0.txt','r') as f:
#     content=f.read()
#     with open('code/1.genome_collection/outputs/sce1900_strain_type_color_strip.txt','r+') as f1:
#         original=f1.read()
#         f1.seek(0)   # move the cursor to the head of file
#         f1.write(content+'\n'+original)

#### 2.color gradient dataset for genome size

In [ ]:
df_genomeSize=all_strain_info[['genome_id','genomeSize']]
df_genomeSize.set_index('genome_id',inplace=True)

# df_genomeSize=df_genomeSize[df_genomeSize.index.isin(strainIDlist)]
# write df_genomeSize behind the code/1.genome_collection/outputs/sce1900_genome_size_strip.txt and use space as separator
df_genomeSize.to_csv('code/1.genome_collection/outputs/sce1900_genome_size_gradient.txt',sep=' ')

with open('data\itol_templates\dataset_gradient_template.txt','r') as f:
    content=f.read()
    with open('code/1.genome_collection/outputs/sce1900_genome_size_gradient.txt','r+') as f1:
        original=f1.read()
        f1.seek(0)   # move the cursor to the head of file
        f1.write(content+'\n'+original)

#### 3. color gradient dataset from proteome size

In [ ]:
df_geneNumb=all_strain_info[['genome_id','gene_numb']]
df_geneNumb.set_index('genome_id',inplace=True)

# df_geneNumb=df_geneNumb[df_geneNumb.index.isin(strainIDlist)]
# only keep the strain with gene number > 5000
df_geneNumb=df_geneNumb[df_geneNumb['gene_numb']>5000]
# write df_geneNumb behind the code/1.genome_collection/outputs/sce1900_gene_numb_gradient.txt and use space as separator
df_geneNumb.to_csv('code/1.genome_collection/outputs/sce1900_gene_numb_gradient.txt',sep=' ')

with open('data\itol_templates\dataset_gradient_template.txt','r') as f:
    content=f.read()
    with open('code/1.genome_collection/outputs/sce1900_gene_numb_gradient.txt','r+') as f1:
        original=f1.read()
        f1.seek(0)   # move the cursor to the head of file
        f1.write(content+'\n'+original)

#### 4.label color dataset for source origin

In [43]:
# label as 1011_strains, new_strains
df_strain_source=all_strain_info[['source']]
df_strain_source['source']=df_strain_source['source'].apply(lambda x: '1011_strains' if x=='1011_nature' else 'new_strains')
# set color: 1011_strains: #407193 ; new_strains: #c2402a
df_strain_source['color']=df_strain_source['source'].apply(lambda x: '#0000ff' if x=='1011_strains' else '#ff0000')
# set id as id|id
df_strain_source['id']=df_strain_source.index
df_strain_source['id']=df_strain_source['id'].apply(lambda x: str(x)+'|'+str(x))

df_strain_source['type']='range'
# remove index
df_strain_source.reset_index(inplace=True)

df_strain_source=df_strain_source[['id','type','color','source']]
df_strain_source

# save and ignore index
df_strain_source.to_csv(r'code/1.genome_collection/outputs/sce1807_strain_source_color_label.txt',sep=' ',index=False)

with open(r'data\itol_templates\colors_styles_template.txt','r') as f:
    content=f.read()
    with open('code/1.genome_collection/outputs/sce1807_strain_source_color_label.txt','r+') as f1:
        original=f1.read()
        f1.seek(0)   # move the cursor to the head of file
        f1.write(content+'\n'+original)

C:\Users\wangh\AppData\Local\Temp\ipykernel_10312\1656911878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strain_source['source']=df_strain_source['source'].apply(lambda x: '1011_strains' if x=='1011_nature' else 'new_strains')
C:\Users\wangh\AppData\Local\Temp\ipykernel_10312\1656911878.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strain_source['color']=df_strain_source['source'].apply(lambda x: '#0000ff' if x=='1011_strains' else '#ff0000')
C:\Users\wangh\AppData\Local\Temp\ipykernel_1

In [ ]:
# # rename lable, 1011_nature:Peter,J.et_al(2018),lg_other:Li,G.et_al(2019),bar_266:Duan,S.et_al(2018),bar_126:Han,D.et_al.(2021),
# # YJM_92:Zhu,Y.et_al.(2016), giga_36:Kang,K.et_al.(2019),lg_others,PRJEB7675:ncbi_PRJEB7675

# df_strain_source.loc[df_strain_source['source']=='1011_nature','source']='Peter,J.et_al(2018)'
# df_strain_source.loc[df_strain_source['source']=='lg_other','source']='Li,G.et_al(2019)'
# df_strain_source.loc[df_strain_source['source']=='bar_266','source']='Duan,S.et_al(2018)'
# df_strain_source.loc[df_strain_source['source']=='bar_126','source']='Han,D.et_al.(2021)'
# df_strain_source.loc[df_strain_source['source']=='YJM_92','source']='Zhu,Y.et_al.(2016)'
# df_strain_source.loc[df_strain_source['source']=='giga_36','source']='Kang,K.et_al.(2019)'
# df_strain_source.loc[df_strain_source['source']=='lg_others','source']='Li,G.et_al(2019)'
# df_strain_source.loc[df_strain_source['source']=='PRJEB7675','source']='ncbi_PRJEB7675'

# # set color for each source：#f44336 #744700 #ce7e00 #8fce00 #2986cc #16537e #6a329f #c90076
# # change itol color to RGB color: rgba(255,0,0,0.5) rgba(0,0,255,0.5) rgba(0,255,255,0.5) rgba(0,255,0,0.5) rgba(192,192,192,0.5) rgba(255,255,0,0.5) rgba(128,42,42,0.5)

# df_strain_source['color']=''
# df_strain_source.loc[df_strain_source['source']=='Peter,J.et_al(2018)','color']='rgba(255,0,0,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Li,G.et_al(2019)','color']='rgba(0,0,255,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Duan,S.et_al(2018)','color']='rgba(0,255,255,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Han,D.et_al.(2021)','color']='rgba(0,255,0,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Zhu,Y.et_al.(2016)','color']='rgba(192,192,192,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Kang,K.et_al.(2019)','color']='rgba(255,255,0,0.5)'
# df_strain_source.loc[df_strain_source['source']=='ncbi_PRJEB7675','color']='rgba(128,42,42,0.5)'
# df_strain_source.loc[df_strain_source['source']=='Li,G.et_al(2019)','color']='rgba(0,0,255,0.5)'

# # set order: source, color
# df_strain_source2=df_strain_source.copy()
# df_strain_source2['label']='label_background'
# df_strain_source2=df_strain_source2[['label','color']]

# # # set all word to lower case
# # df_strain_source2['color']=df_strain_source2['color'].str.lower()

# # write df_strain_source behind the code/1.genome_collection/outputs/sce1900_strain_source_color_strip.txt and use space as separator
# df_strain_source2.to_csv('code/1.genome_collection/outputs/sce1900_strain_source_color_label.txt',sep=' ')

# with open(r'data\itol_templates\colors_styles_template.txt','r') as f:
#     content=f.read()
#     with open('code/1.genome_collection/outputs/sce1900_strain_source_color_label.txt','r+') as f1:
#         original=f1.read()
#         f1.seek(0)   # move the cursor to the head of file
#         f1.write(content+'\n'+original)